We assume that we sill start with the working copy

In [2]:
#Importing libraries
import pandas as pd
import re
import datetime

In [3]:
today_date = datetime.date.today()
print(today_date)

2024-08-15


In [4]:
def clean_special_and_split(text):
    
    #Checking if str, if not making it str
    if not isinstance(text, str):
        text = str(text)
        
    #Replacing - and + by space
    text = text.replace('-', ' ').replace('+', ' ')
    
    #Removing all but dots between letters
    text_cleaned = re.sub(r'[^a-zA-Z0-9.\s]', '', text)
    
    #Replacing dots between letters by space
    text_cleaned = re.sub(r'(?<=\w)\.(?=\w)', ' ', text_cleaned)
    
    text_cleaned = text_cleaned.upper()
    words = text_cleaned.split()
    
    return words

In [5]:
def compare_cells(row):
    a = clean_special_and_split(row['Manufacturer_implementation_clean'])
    b = clean_special_and_split(row['Manufacturer_client_clean'])
    if any(word in a for word in b):
        return 1
    else:
        return 0

In [6]:
df = pd.read_excel("RR Inchinnan SnapPart Dataload v3.xlsx")
df_client = df[["Item  Code", "Manufacturer"]]
df_client.rename(columns = {"Item  Code" : "Item_code", "Manufacturer" : "Manufacturer_implementation"}, inplace= True)
df_client.head()

C:\Users\misol\AppData\Local\Temp\ipykernel_15164\1927256225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_client.rename(columns = {"Item  Code" : "Item_code", "Manufacturer" : "Manufacturer_implementation"}, inplace= True)


,Item_code,Manufacturer_implementation
0,3HAC16831-1,ABB Limited
1,535-4346,ABB Limited
2,DSQC-136,ABB Limited
3,S830C-D125,ABB Limited
4,5354302,ABB Limited


In [23]:
implementation_input_file = input('name of the implementation input file :')
df = pd.read_excel(f"{implementation_input_file}.xlsx")
df_implementation = df[["Item  Code", "Manufacturer"]]
df_implementation.rename(columns = {"Item  Code" : "Item_code", "Manufacturer" : "Manufacturer_implementation"}, inplace= True)
df_implementation.head()

C:\Users\misol\AppData\Local\Temp\ipykernel_10000\3940454555.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_implementation.rename(columns = {"Item  Code" : "Item_code", "Manufacturer" : "Manufacturer_implementation"}, inplace= True)


,Item_code,Manufacturer_implementation
0,CRR-016098,HYD022
1,CRR-016106,HYD022
2,CRR-016030,HYD022
3,ZSACCTX,CAMS CARDIFF
4,ZCK-Y13,TBD - To Be Determined


In [24]:
df_merged = pd.merge(df_client, df_implementation, on="Item_code", how="inner")

df_merged = df_merged.sort_values(by="Item_code")
df_merged.dropna(inplace=True)
df_merged['match'] = 0

In [25]:
df_merged.loc[:, 'Manufacturer_client'] = df_merged['Manufacturer_client']
df_merged.loc[:, 'Manufacturer_client_clean'] = df_merged['Manufacturer_client'].apply(clean_special_and_split)

df_merged.loc[:, 'Manufacturer_implementation'] = df_merged['Manufacturer_implementation']
df_merged.loc[:, 'Manufacturer_implementation_clean'] = df_merged['Manufacturer_implementation'].apply(clean_special_and_split)

df_merged['match'] = df_merged.apply(compare_cells, axis=1)

df_merged.head()

,Item_code,Manufacturer_client,Manufacturer_implementation,match,Manufacturer_client_clean,Manufacturer_implementation_clean
1,#234 9972-1100,RETECH,RETECH,1,[RETECH],[RETECH]
2,#237 #720,TBD - To Be Determined,TBD - To Be Determined,1,"[TBD, TO, BE, DETERMINED]","[TBD, TO, BE, DETERMINED]"
3,#242-9972-630,RETECH,RETECH,1,[RETECH],[RETECH]
4,#250 2130-BNA-250,RETECH,RETECH,1,[RETECH],[RETECH]
5,#341,RETECH,RETECH,1,[RETECH],[RETECH]


In [26]:
df_merged.to_excel(f'RRB_match_{today_date}.xlsx', sheet_name='Feuille1', index=False)